In [ ]:
def get_client_iqn():
    print("클라이언트 IQN 확인 중...")
    stdout = cc("cat /etc/iscsi/initiatorname.iscsi")
    for line in stdout:
        line = line.strip()
        if line.startswith("InitiatorName="):
            iqn = line.split("=")[1]
            print(f"클라이언트 IQN: {iqn}")
            return iqn
    print("IQN을 확인할 수 없습니다.")
    return None

# iSCSI Target discovery 및 로그인
def iscsi_login(target_ip="172.16.27.175", target_port=3260):  #서버의 IP
    print(f"iSCSI 타겟 {target_ip} 검색 중...")
    stdout = cc(f"iscsiadm -m discovery -t sendtargets -p {target_ip}:{target_port}")
    pp(stdout)

    print(f"iSCSI 타겟 {target_ip} 로그인 중...")
    stdout = cc(f"iscsiadm -m node -T {target_ip}:3260 -l")
    pp(stdout)
    time.sleep(2)

# 디스크 확인
def list_new_disks():
    print("새로운 디스크 확인 중...")
    stdout = cc("lsblk -dn -o NAME,SIZE,TYPE | grep disk")
    disks = [line.strip() for line in stdout]
    pp(stdout)
    return disks

# 파티션 생성 및 포맷
def create_and_format_partition():
    disks = list_new_disks()
    for disk in disks:
        disk_name = disk.split()[0]
        print(f"\n디스크 /dev/{disk_name} 파티션 및 포맷 진행...")
        cc(f"echo -e 'o\nn\np\n1\n\n\nw' | fdisk /dev/{disk_name}")
        cc(f"mkfs.ext4 /dev/{disk_name}1")
        mount_point = f"/mnt/{disk_name}1"
        cc(f"mkdir -p {mount_point}")
        cc(f"mount /dev/{disk_name}1 {mount_point}")
        print(f"/dev/{disk_name}1 -> {mount_point} 마운트 완료")